# 01_test

Se realizará fine-tuning al modelo `gpt-3.5-turbo`.

Los comandos me funcionaron con la versión 0.28.0 de la librería `openai`.

In [1]:
# pip install openai
# pip install tiktoken 
# pip install langchain

Importar librerías.

In [4]:
import os
import openai

Importar API key.

In [3]:
os.environ["OPENAI_API_KEY"] = "..." # Reemplazar los 3 puntos por tu propia API key.

Cargar datos.

La archivo contiente una serie de preguntas y respuestas, en donde estas últimas tratan de simular lo que respondería el personaje.

In [1]:
with open("dialogos_burro.txt") as f: 
    text = [line for line in f]

In [4]:
text[: 10]

['user: ¡Puedes hablar!\n',
 'assistant: ¡Así es tonto! Ahora soy un burro que habla y vuela ¿Han visto como su dinero vuela? ¡¿O a Caperucita y la Abuela?! ¡Pero a que nunca han visto cómo un burro vuela! Jajajaja\n',
 '-\n',
 'user: ¿Estás hablando con...migo?\n',
 'assistant: ¡Claro! Hablaba contigo. Oye, ¡Estuviste enorme! Esos cuates me querían como burro de carga. Pero llegaste así "¡Bam!" patitas pa\' que las quiero. Se jueron de volada. Fue muy chistoso verlos correr.\n',
 '-\n',
 'user: ¿Ahora, Por qué no te vas a celebrar tu libertad con tus amigos? \n',
 'assistant: Es que… Yo no tengo amigos. Y no pienso ir al bosque yo solito. Hey! Tengo una ideota. Me quedaré contigo. Tu eres verdaderamente una máquina de pelea. Haremos tronar a cualquiera.\n',
 '-\n',
 'user: Y se te hago un rugido así de gigante! GRRRRRUAUUUUU!!!\n']

Aplicar formato necesario.

Ahora debemos asegurarnos que cada ejemplo siga el siguiente formato:

```
{
  "messages": [
    { "role": "system", "content": "You are an assistant that occasionally misspells words" },
    { "role": "user", "content": "Tell me a story." },
    { "role": "assistant", "content": "One day a student went to schoool." }
  ]
}
```

Luego se debe programar una función que construya cada ejemplo como un diccionario con una única llave `messages` y cuyo valor es el mensaje del sistema, más la conversación entre usuario y asistente.

In [6]:
def formatear_ejemplo(lista_mensajes, system_message = None):
    messages = []

    # Incluir primero el mensaje de sistema.
    if system_message:
        messages.append({
            "role": "system",
            "content": system_message
        })

    # Iterar por la lista de mensajes.
    for mensaje in lista_mensajes:
        # Separar los mensajes por los dos puntos y el espacio.
        partes = mensaje.split(": ", maxsplit = 1)

        #Controlar si alguna línea no cumple el patrón.
        if len(partes) < 2:
            continue

        # Identificar role y content.
        role = partes[0].strip()
        content = partes[1].strip()

        # Formatear el mensaje.
        message = {
            "role": role,
            "content": content
        }

        # Agregar el mensaje a la lista.
        messages.append(message)

    # Crear diccionario final.
    dict_final = {
        "messages": messages
    }

    return dict_final

Aplicar la función a cada ejemplo.

In [7]:
system_message = "Eres un Burro muy parlanchín y muy ingenioso en tus respuestas. \
Usa los símbolos [ y ] para señalar que realizas alguna acción. \
Por ejemplo, para señalar que extiendes la mano: \
Hola, ¿cómo estás? [extiendo la mano]."

dataset = []

ejemplo = []

for line in text:
  if line == "-\n":
    ejemplo_formateado = formatear_ejemplo(lista_mensajes = ejemplo, system_message = system_message)
    dataset.append(ejemplo_formateado)
    ejemplo = []
    continue

  ejemplo.append(line)

In [8]:
dataset[4]

{'messages': [{'role': 'system',
   'content': 'Eres un Burro muy parlanchín y muy ingenioso en tus respuestas. Usa los símbolos [ y ] para señalar que realizas alguna acción. Por ejemplo, para señalar que extiendes la mano: Hola, ¿cómo estás? [extiendo la mano].'},
  {'role': 'user', 'content': '¿Por qué estás siguiéndome?'},
  {'role': 'assistant',
   'content': "Te diré por qué. Porque 'toy solito, no hay nadie aquí a mi lado. No habrá problemas hoy. De mí ya se han burlado. Amigos debes tener!"}]}

Validar formato, errores y estimar precio.

Revisamos si hay errores y estimamos el precio usando la [guía entregada por OpenAI](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset).

[Pricing](https://openai.com/pricing).

![](./media/pricing.png)

In [9]:
# Encontrar errores de formato.

from collections import defaultdict

format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Errores encontrados:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No se encontraron errores")

Errores encontrados:
missing_content: 2


In [10]:
# tiktoken

import tiktoken
import numpy as np

encoding = tiktoken.get_encoding("cl100k_base")

# Método simplificado de https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb.

def num_tokens_from_messages(messages, tokens_per_message = 3, tokens_per_name = 1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribución de {name}:")
    print(f"min, max: {min(values)}, {max(values)}")
    print(f"media, mediana: {np.mean(values)}, {np.median(values)}")
    print(f"p5, p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [11]:
# Por último, podemos ver los resultados de las diferentes operaciones de formateo antes de continuar con la creación de un trabajo de ajuste:

# Advertencias y recuentos de tokens:
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("N° de ejemplos sin el system message:", n_missing_system)
print("N° de ejemplos sin el user message:", n_missing_user)
print_distribution(n_messages, "num_mensajes_por_ejemplo")
print_distribution(convo_lens, "num_total_tokens_por_ejemplo")
print_distribution(assistant_message_lens, "num_assistant_tokens_por_ejemplo")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} ejemplos que excedan el límite de tokenes de 4096, ellos serán truncados durante el fine-tuning.")

N° de ejemplos sin el system message: 0
N° de ejemplos sin el user message: 0

#### Distribución de num_mensajes_por_ejemplo:
min, max: 3, 4
media, mediana: 3.007518796992481, 3.0
p5, p95: 3.0, 3.0

#### Distribución de num_total_tokens_por_ejemplo:
min, max: 89, 244
media, mediana: 123.96992481203007, 118.0
p5, p95: 97.0, 162.0

#### Distribución de num_assistant_tokens_por_ejemplo:
min, max: 2, 123
media, mediana: 25.69924812030075, 18.0
p5, p95: 6.0, 57.599999999999994

0 ejemplos que excedan el límite de tokenes de 4096, ellos serán truncados durante el fine-tuning.


In [12]:
# Precios y estimación predeterminada de n_epochs.

MAX_TOKENS_PER_EXAMPLE = 4096
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 4
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)

if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)

print(f"El conjunto de datos tiene {n_billing_tokens_in_dataset} tokens que serán cargados durante el entrenamiento.")
print(f"Por defecto, entrenarás para {n_epochs} epochs en este conjunto de datos.")
print(f"Por defecto, serás cargado con {n_epochs * n_billing_tokens_in_dataset} tokens.")
print("Revisa la página para estimar el costo total.")

El conjunto de datos tiene 16488 tokens que serán cargados durante el entrenamiento.
Por defecto, entrenarás para 4 epochs en este conjunto de datos.
Por defecto, serás cargado con 65952 tokens.
Revisa la página para estimar el costo total.


Guardar base de datos en JSONL.

In [11]:
import json

def save_to_jsonl(dataset, file_path):
    with open(file_path, 'w') as file:
        for ejemplo in dataset:
            json_line = json.dumps(ejemplo)
            file.write(json_line + '\n')

In [12]:
# Guardar train_full.jsonl

save_to_jsonl(dataset, 'dialogos_burro_train_full.jsonl')

Subir archivos.

Cargamos la base de datos a los servidores de OpenAI y luego imprimimos el `id` de la respuesta de esta solicitd. Hacemos esto porque vamos a necesitar el id posteriormente.

In [13]:
train_full_response_file = openai.File.create(
    file = open('dialogos_burro_train_full.jsonl', 'rb'),
    purpose = 'fine-tune'
)

print(f'id: {train_full_response_file.id}')

id: file-3Vuw06RwrQqwZnd2C6h1Zw7P


**CUIDADO AL CORRER (GASTA CRÉDITOS)**

Luego creamos un punto de trabajo para hacer fine-tuning.


In [15]:
# train_full_response_file.id: file-3Vuw06RwrQqwZnd2C6h1Zw7P

response = openai.FineTuningJob.create(training_file = train_full_response_file.id,
                                       model = "gpt-3.5-turbo",
                                       suffix = 'burro-shrek',
                                       hyperparameters = {'n_epochs': 4})

In [16]:
response

<FineTuningJob fine_tuning.job id=ftjob-YhehHcFnX4JPfgfzIwLPKWP7 at 0x28ac9500eb0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-YhehHcFnX4JPfgfzIwLPKWP7",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1705329629,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-NuAQNfbr64DNxg1qLIRYgWlU",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-3Vuw06RwrQqwZnd2C6h1Zw7P",
  "hyperparameters": {
    "n_epochs": 4,
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [17]:
openai.FineTuningJob.retrieve(response.id)

<FineTuningJob fine_tuning.job id=ftjob-YhehHcFnX4JPfgfzIwLPKWP7 at 0x28ac95cd7c0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-YhehHcFnX4JPfgfzIwLPKWP7",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1705329629,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-NuAQNfbr64DNxg1qLIRYgWlU",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-3Vuw06RwrQqwZnd2C6h1Zw7P",
  "hyperparameters": {
    "n_epochs": 4,
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [18]:
response = openai.FineTuningJob.list_events(id = response.id)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Created fine-tuning job: ftjob-YhehHcFnX4JPfgfzIwLPKWP7
Validating training file: file-3Vuw06RwrQqwZnd2C6h1Zw7P


Pruebas.

Esperamos a que llegue el correo de confirmación, donde nos entregarán el `id` del nuevo modelo entrenado. Usaremos `langchain`.

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

# Prueba el modelo "fine-tuneado".

model_name = "ft:gpt-3.5-turbo-0613:melitacruces:burro-shrek:8hItgra6"
chat = ChatOpenAI(model = model_name, temperature = 0.0)

messages = [
    SystemMessage(content = system_message),
    HumanMessage(content = "Hola, que lindo día.") # Reemplazar por tu propio mensaje.
]

response = chat(messages)
print(response.content)

Sí, muy lindo. Sol radiante, cielo azul... chido.


In [27]:
# Prueba el modelo sin ajustar o puedes reemplazarlo por el modelo que desees (de GPT).

chat = ChatOpenAI(model = 'gpt-3.5-turbo', temperature = 0.0)

messages = [
    SystemMessage(content = system_message),
    HumanMessage(content = "Hola, que lindo día.")
]

response = chat(messages)
print(response.content)

¡Hola! ¡Sí, es un día maravilloso! ¿En qué puedo ayudarte hoy?
